
Basic transfer learning with cats and dogs data

https://github.com/sasidhar-programmer/Tensorflow_Advance_Techniques/blob/main/3-Advance_computer_vision/week%20-1/C3_W1_Lab_1_transfer_learning_cats_dogs.ipynb

In [1]:
import urllib.request
import os
import zipfile
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
from shutil import copyfile

In [2]:
# data_url = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip"
# data_file_name = "catsdogs.zip"
# download_dir = '../dataset/'
# urllib.request.urlretrieve(data_url, data_file_name)
# zip_ref = zipfile.ZipFile(data_file_name, 'r')
# zip_ref.extractall(download_dir)
# zip_ref.close()

In [3]:


print("Number of cat images:",len(os.listdir('../dataset/PetImages/Cat/')))
print("Number of dog images:", len(os.listdir('../dataset/PetImages/Dog/')))




Number of cat images: 12501
Number of dog images: 12501


In [4]:
# try:
#     os.mkdir('../dataset/cats-v-dogs')
#     os.mkdir('../dataset/cats-v-dogs/training')
#     os.mkdir('../dataset/cats-v-dogs/testing')
#     os.mkdir('../dataset/cats-v-dogs/training/cats')
#     os.mkdir('../dataset/cats-v-dogs/training/dogs')
#     os.mkdir('../dataset/cats-v-dogs/testing/cats')
#     os.mkdir('../dataset/cats-v-dogs/testing/dogs')
# except OSError:
#     pass



In [5]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[:testing_length]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)

In [6]:
CAT_SOURCE_DIR = "../dataset/PetImages/Cat/"
TRAINING_CATS_DIR = "../dataset/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "../dataset/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "../dataset/PetImages/Dog/"
TRAINING_DOGS_DIR = "../dataset/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "../dataset/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

# Expected output
# 666.jpg is zero length, so ignoring
# 11702.jpg is zero length, so ignoring

666.jpg is zero length, so ignoring.
11702.jpg is zero length, so ignoring.


In [7]:
print("Number of training cat images", len(os.listdir('../dataset/cats-v-dogs/training/cats/')))
print("Number of training dog images", len(os.listdir('../dataset/cats-v-dogs/training/dogs/')))
print("Number of testing cat images", len(os.listdir('../dataset/cats-v-dogs/testing/cats/')))
print("Number of testing dog images", len(os.listdir('../dataset/cats-v-dogs/testing/dogs/')))

Number of training cat images 11250
Number of training dog images 11250
Number of testing cat images 1250
Number of testing dog images 1250


In [8]:


TRAINING_DIR = "../dataset/cats-v-dogs/training/"
# Experiment with your own parameters to reach 99.9% validation accuracy or better
train_datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=100,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

VALIDATION_DIR = "../dataset/cats-v-dogs/testing/"

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=100,
                                                              class_mode='binary',
                                                              target_size=(150, 150))



Found 22498 images belonging to 2 classes.
Found 2499 images belonging to 2 classes.
